# Tahu lah apa ini?

In [3]:
# Path to google drive folder
# %cd /content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002. Laporan-eksperimen
# %cd /content/drive/MyDrive/Colab Notebooks/Disertasi-Ahmad-Rofiqul/002.Experiment_SWI_16052024/

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest, VotingClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from pyod.models.hbos import HBOS
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, OutlierMixin
from scipy.spatial.distance import mahalanobis

In [4]:
# Load the dataset
# file_path = '01.Dataset-Rofiqul/agriculture_dataset.csv'
# file_path = '01.Dataset-Rofiqul/Ujicoba3-29052024/agriculture_dataset.csv'
# file_path = 'dataset/agriculture_dataset.csv'
# file_path = 'dataset/09_05_1_AgricultureOutliersResult_30052024.csv'
file_path = 'dataset/09_05_3_0_AgricultureOutliersDetectionResults.csv'
unfiltered = pd.read_csv(file_path)
unfiltered

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,lof_outlier,ocsvm_outlier,elliptic_outlier,hbos_outlier,dbscan_outlier,mahalanobis_outlier,iqr_outlier,zscore_outlier,optics_outlier,cblof_outlier
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,1,1,1,1,1,1,1,1,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,1,1,1,1,1,1,1,1,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,1,1,1,1,1,1,1,1,-1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,1,1,1,1,1,1,1,1,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,1,1,1,1,1,1,1,1,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,1,1,1,1,1,1,1,1,-1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,1,1,1,1,1,1,1,1,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,1,1,1,1,1,1,1,1,1,1


In [5]:
# Define columns to keep
columns_to_keep = ['Date', 'Year', 'Experiment', 'DataUse', 'Replication', 'Month',
                   'Vegetation', 'VegType', 'N2O', 'N_rate', 'PP2', 'PP7', 'AirT',
                   'DAF_TD', 'DAF_SD', 'WFPS25cm', 'NH4', 'NO3', 'Clay', 'Sand',
                   'SOM', 'if_outlier', 'lof_outlier']

# Filter the DataFrame to keep only the specified columns
data = unfiltered[columns_to_keep]

In [6]:

# Display the first few rows of the dataset for review
# data.head(),data.info(),data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         2246 non-null   object 
 1   Year         2246 non-null   int64  
 2   Experiment   2246 non-null   object 
 3   DataUse      2246 non-null   object 
 4   Replication  2246 non-null   object 
 5   Month        2246 non-null   object 
 6   Vegetation   2246 non-null   object 
 7   VegType      2246 non-null   object 
 8   N2O          2246 non-null   float64
 9   N_rate       2246 non-null   int64  
 10  PP2          2246 non-null   float64
 11  PP7          2246 non-null   float64
 12  AirT         2246 non-null   float64
 13  DAF_TD       2246 non-null   int64  
 14  DAF_SD       2246 non-null   int64  
 15  WFPS25cm     2246 non-null   float64
 16  NH4          2246 non-null   float64
 17  NO3          2246 non-null   float64
 18  Clay         2246 non-null   float64
 19  Sand  

In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.base import BaseEstimator, ClassifierMixin


In [8]:
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,DAF_TD,DAF_SD,WFPS25cm,NH4,NO3,Clay,Sand,SOM,if_outlier,lof_outlier
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,276,241,0.666508,11.046340,22.940812,62.500000,637.500000,1.174072,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,277,242,0.640608,11.008087,22.959578,62.500000,637.500000,1.174072,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,285,250,0.728085,10.831669,23.221928,62.500000,637.500000,1.174072,1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,286,251,0.686872,10.849792,23.271978,62.500000,637.500000,1.174072,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,312,277,0.716221,10.204748,24.206855,62.500000,637.500000,1.174072,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,167,167,0.390203,17.061793,5.803681,128.333333,491.666667,1.806000,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,197,197,0.317040,5.624730,5.429873,162.500000,490.375000,1.496400,1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,197,197,0.341428,3.504328,7.239000,183.250000,432.875000,1.186800,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,197,197,0.365815,3.636931,8.434294,169.833333,418.833333,1.823200,1,1


In [9]:

# Generate synthetic ground truth labels for evaluation purposes
# Creating a synthetic ground truth based on majority voting of outliers
# models=['if_outlier', 'lof_outlier','cblof_outlier', 'ocsvm_outlier', 'elliptic_outlier', 'hbos_outlier', 'dbscan_outlier', 'mahalanobis_outlier','zscore_outlier','optics_outlier','iqr_outlier']
models=['if_outlier', 'lof_outlier']
data['majority_voting'] = data.apply(lambda row: 1 if row[models].mode()[0] == 1 else -1, axis=1)
# Generate hard voting label
# data['hard_voting'] = data[models].sum(axis=1).apply(lambda x: -1 if x < 0 else 1)

# import numpy as np


# Append the outlier results to the original data
# data = pd.concat([data, data[['IF_outlier', 'LOF_Outlier', 'OCSVM_Outlier', 'Elliptic_Outlier', 'HBOS_Outlier', 'DBSCAN_Outlier', 'mahalanobis_Outlier', 'Majority_Voting']]], axis=1)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16204\4286098094.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['majority_voting'] = data.apply(lambda row: 1 if row[models].mode()[0] == 1 else -1, axis=1)


In [10]:
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,DAF_SD,WFPS25cm,NH4,NO3,Clay,Sand,SOM,if_outlier,lof_outlier,majority_voting
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,241,0.666508,11.046340,22.940812,62.500000,637.500000,1.174072,1,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,242,0.640608,11.008087,22.959578,62.500000,637.500000,1.174072,1,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,250,0.728085,10.831669,23.221928,62.500000,637.500000,1.174072,1,1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,251,0.686872,10.849792,23.271978,62.500000,637.500000,1.174072,1,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,277,0.716221,10.204748,24.206855,62.500000,637.500000,1.174072,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,167,0.390203,17.061793,5.803681,128.333333,491.666667,1.806000,1,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,197,0.317040,5.624730,5.429873,162.500000,490.375000,1.496400,1,1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,197,0.341428,3.504328,7.239000,183.250000,432.875000,1.186800,1,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,197,0.365815,3.636931,8.434294,169.833333,418.833333,1.823200,1,1,1


In [167]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             2246 non-null   object 
 1   Year             2246 non-null   int64  
 2   Experiment       2246 non-null   object 
 3   DataUse          2246 non-null   object 
 4   Replication      2246 non-null   object 
 5   Month            2246 non-null   object 
 6   Vegetation       2246 non-null   object 
 7   VegType          2246 non-null   object 
 8   N2O              2246 non-null   float64
 9   N_rate           2246 non-null   int64  
 10  PP2              2246 non-null   float64
 11  PP7              2246 non-null   float64
 12  AirT             2246 non-null   float64
 13  DAF_TD           2246 non-null   int64  
 14  DAF_SD           2246 non-null   int64  
 15  WFPS25cm         2246 non-null   float64
 16  NH4              2246 non-null   float64
 17  NO3           

In [168]:
# Creating a synthetic ground truth based on majority voting of outliers
# data['majority'] = data.apply(lambda row: 1 if row[['if_outlier','lof_outlier','ocsvm_outlier','elliptic_outlier','hbos_outlier','dbscan_outlier','mahalanobis_outlier']].mode()[0] == 1 else -1, axis=1)
# Convert outlier labels to binary (1 for outliers, 0 for inliers) for evaluation metrics
# y_true = (data['majority_voting'] == -1).astype(int)

In [169]:
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,NH4,NO3,Clay,Sand,SOM,if_outlier,dbscan_outlier,ocsvm_outlier,majority_voting,hard_voting
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,11.046340,22.940812,62.500000,637.500000,1.174072,1,1,1,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,11.008087,22.959578,62.500000,637.500000,1.174072,1,1,1,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,10.831669,23.221928,62.500000,637.500000,1.174072,1,1,1,1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,10.849792,23.271978,62.500000,637.500000,1.174072,1,1,1,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,10.204748,24.206855,62.500000,637.500000,1.174072,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,17.061793,5.803681,128.333333,491.666667,1.806000,1,1,1,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,5.624730,5.429873,162.500000,490.375000,1.496400,1,1,1,1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,3.504328,7.239000,183.250000,432.875000,1.186800,1,1,1,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,3.636931,8.434294,169.833333,418.833333,1.823200,1,1,1,1,1


In [11]:


# Voting-based with averaging (soft voting)
# data['average_voting'] = data[[f'{name}_outlier' for name, _ in models]].mean(axis=1).apply(lambda x: -1 if x < 0 else 1)
data['average_voting'] = data[models].mean(axis=1).apply(lambda x: -1 if x < 0 else 1)
# data['soft_voting'] = (data[models] == -1).sum(axis=1) > 1


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16204\1849030082.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['average_voting'] = data[models].mean(axis=1).apply(lambda x: -1 if x < 0 else 1)


In [12]:
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,WFPS25cm,NH4,NO3,Clay,Sand,SOM,if_outlier,lof_outlier,majority_voting,average_voting
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,0.666508,11.046340,22.940812,62.500000,637.500000,1.174072,1,1,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,0.640608,11.008087,22.959578,62.500000,637.500000,1.174072,1,1,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,0.728085,10.831669,23.221928,62.500000,637.500000,1.174072,1,1,1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,0.686872,10.849792,23.271978,62.500000,637.500000,1.174072,1,1,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,0.716221,10.204748,24.206855,62.500000,637.500000,1.174072,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,0.390203,17.061793,5.803681,128.333333,491.666667,1.806000,1,1,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,0.317040,5.624730,5.429873,162.500000,490.375000,1.496400,1,1,1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,0.341428,3.504328,7.239000,183.250000,432.875000,1.186800,1,1,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,0.365815,3.636931,8.434294,169.833333,418.833333,1.823200,1,1,1,1


In [13]:
# Weighted voting
weights = {
    'if_outlier': 1,
    # 'dbscan_outlier': 1,
    # 'ocsvm_outlier': 1,
    'lof_outlier': 1,
    # 'elliptic_outlier': 1,
    # 'hbos_outlier': 1,
    # # 'mahalanobis_outlier': 1,
    # 'iqr_outlier':1,
    # 'zscore_outlier':1,
    # 'cblof_outlier':1,
    # 'optics_outlier':1
}
# data['weighted_voting'] = data[[f'{name}_outlier' for name, _ in models]].apply(
# data['weighted_voting'] = data[[models]].apply(
    # lambda row: np.average(row, weights=[weights[name] for name, _ in models]), axis=1).apply(lambda x: -1 if x < 0 else 1)
    # lambda row: np.average(row, weights=[weights[name] for name, _ in models]), axis=1).apply(lambda x: -1 if x < 0 else 1)
data['weighted_voting'] = data[models].apply(
    lambda row: np.average(row, weights=[weights[name] for name in models]), axis=1).apply(lambda x: -1 if x < 0 else 1)

# Calculate the weighted voting process details
# weighted_voting_details = data[models].apply(
    # lambda row: np.average(row, weights=[weights[name] for name in models]), axis=1)

# Add the weighted voting details to the DataFrame
# data['weighted_voting'] = weighted_voting_details.apply(lambda x: -1 if x < 0 else 1)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16204\3681853845.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weighted_voting'] = data[models].apply(


In [14]:
data['weighted_voting']

0       1
1       1
2       1
3       1
4       1
       ..
2241    1
2242    1
2243    1
2244    1
2245    1
Name: weighted_voting, Length: 2246, dtype: int64

In [15]:
# data.to_csv('dataset/09_05_4_AgricultureOutliersResult_01062024_IDO_ONLY.csv', index=False)
# Save the updated data to the CSV file
# data.to_csv('dataset/09_05_4_0_AgricultureOutliersResult_01062024_IDO_ONLY.csv', index=False)
# Save the updated data with outlier detection results
data.to_csv('dataset/09_05_4_1_AgricultureOutliersVotingBasedResult_IF_LOF_ONLY.csv', index=False)

In [16]:
data

,Date,Year,Experiment,DataUse,Replication,Month,Vegetation,VegType,N2O,N_rate,...,NH4,NO3,Clay,Sand,SOM,if_outlier,lof_outlier,majority_voting,average_voting,weighted_voting
0,2/9/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.896742,170,...,11.046340,22.940812,62.500000,637.500000,1.174072,1,1,1,1,1
1,2/10/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,2.190218,170,...,11.008087,22.959578,62.500000,637.500000,1.174072,1,1,1,1,1
2,2/18/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.542594,170,...,10.831669,23.221928,62.500000,637.500000,1.174072,1,1,1,1,1
3,2/19/12,2012,BCSE_KBS,Building,R1,February,Corn,Annual,3.342870,170,...,10.849792,23.271978,62.500000,637.500000,1.174072,1,1,1,1,1
4,3/16/12,2012,BCSE_KBS,Building,R1,March,Corn,Annual,2.947778,170,...,10.204748,24.206855,62.500000,637.500000,1.174072,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,10/16/2013,2013,MCSE-T2,Testing,R4,October,TRIAE,Annual,1.640000,91,...,17.061793,5.803681,128.333333,491.666667,1.806000,1,1,1,1,1
2242,11/15/2013,2013,MCSE-T2,Testing,R1,November,TRIAE,Annual,0.990000,91,...,5.624730,5.429873,162.500000,490.375000,1.496400,1,1,1,1,1
2243,11/15/2013,2013,MCSE-T2,Testing,R2,November,TRIAE,Annual,-0.200000,91,...,3.504328,7.239000,183.250000,432.875000,1.186800,1,1,1,1,1
2244,11/15/2013,2013,MCSE-T2,Testing,R3,November,TRIAE,Annual,0.300000,91,...,3.636931,8.434294,169.833333,418.833333,1.823200,1,1,1,1,1


In [17]:
# Generate statistics for each detection method and voting-based result
methods = [
'if_outlier', 
# 'dbscan_outlier', 
# 'ocsvm_outlier', 
'lof_outlier', 
# 'elliptic_outlier', 
# 'hbos_outlier', 
# 'mahalanobis_outlier',
# 'cblof_outlier',
# 'zscore_outlier',
# 'optics_outlier',
# 'iqr_outlier', 
'average_voting',
'weighted_voting',
'majority_voting',
# 'hard_voting'
]

statistics = {}

for method in methods:
    counts = data[method].value_counts()
    # stats['method']=method
    stats = {
        # 'Method':method,
        'Inliers': counts.get(1, 0),
        'Outliers': counts.get(-1, 0),
        'Total': counts.sum()
    }
    statistics[method] = stats

# Convert the statistics to a DataFrame for better visualization
statistics_df = pd.DataFrame(statistics).T
statistics_df.to_csv('dataset/09_05_4_2_AgricultureOutliersVotingBasedStatsSummary_IF_LOF_ONLY.csv', index=True)
statistics_df

,Inliers,Outliers,Total
if_outlier,2021,225,2246
lof_outlier,2072,174,2246
average_voting,2172,74,2246
weighted_voting,2172,74,2246
majority_voting,1921,325,2246


In [18]:
# Summary of results
summary_table = pd.DataFrame({
    'Method': ['Isolation Forest', 'LOF','Majority Voting','Average Voting'],
    'Outlier Count': [
        (data['if_outlier'] == -1).sum(),
        (data['lof_outlier'] == -1).sum(),
        # (data['dbscan_outlier'] == -1).sum(),
        # (data['ocsvm_outlier'] == -1).sum(),
        (data['majority_voting'] == -1).sum(),
        (data['average_voting'] == -1).sum()
    ],
    'Inlier Count': [
        (data['if_outlier'] == 1).sum(),
        (data['lof_outlier'] == 1).sum(),
        # (data['dbscan_outlier'] != -1).sum(),
        # (data['ocsvm_outlier'] == 1).sum(),
        (data['majority_voting'] == 1).sum(),
        (data['average_voting'] == 1).sum()
    ]
})
summary_table

,Method,Outlier Count,Inlier Count
0,Isolation Forest,225,2021
1,LOF,174,2072
2,Majority Voting,325,1921
3,Average Voting,74,2172


In [181]:
soft = data[(data['average_voting'] == 1)]
hard = data[(data['majority_voting'] == 1)]

In [183]:
hard.to_csv('dataset/09_05_4_1_AgricultureOutliers_HardVotingBased_IDO.csv', index=False)
soft.to_csv('dataset/09_05_4_2_AgricultureOutliers_SoftVotingBased_IDO.csv', index=False)


In [184]:
# Count the number of outliers and non-outliers for each voting method
hard_outlier_count = hard[hard['majority_voting'] == -1].shape[0]
hard_non_outlier_count = hard[hard['majority_voting'] == 1].shape[0]

soft_outlier_count = soft[soft['average_voting'] == -1].shape[0]
soft_non_outlier_count = soft[soft['average_voting'] == 1].shape[0]
print(f"Hard Voting - Number of outliers: {hard_outlier_count}")
print(f"Hard Voting - Number of non-outliers: {hard_non_outlier_count}")

print(f"Soft Voting - Number of outliers: {soft_outlier_count}")
print(f"Soft Voting - Number of non-outliers: {soft_non_outlier_count}")

Hard Voting - Number of outliers: 0
Hard Voting - Number of non-outliers: 2072
Soft Voting - Number of outliers: 0
Soft Voting - Number of non-outliers: 2072


In [185]:
# Summary of outliers detection
outlier_summary = {
    'Method': ['Isolation Forest', 'DBSCAN', 'One-Class SVM'],
    'Outliers': [
        np.sum(hard['if_outlier'] == -1),
        np.sum(hard['dbscan_outlier'] == -1),
        np.sum(hard['ocsvm_outlier'] == -1),
    ],
    'Inliers': [
        np.sum(hard['if_outlier'] == 1),
        np.sum(hard['dbscan_outlier'] == 1),
        np.sum(hard['ocsvm_outlier'] == 1),
    ]
}
summary_df = pd.DataFrame(outlier_summary)
summary_df

,Method,Outliers,Inliers
0,Isolation Forest,51,2021
1,DBSCAN,0,2072
2,One-Class SVM,50,2022


In [186]:
# Summary of outliers detection
outlier_summary = {
    'Method': ['Isolation Forest', 'DBSCAN', 'One-Class SVM'],
    'Outliers': [
        np.sum(soft['if_outlier'] == -1),
        np.sum(soft['dbscan_outlier'] == -1),
        np.sum(soft['ocsvm_outlier'] == -1),
    ],
    'Inliers': [
        np.sum(soft['if_outlier'] == 1),
        np.sum(soft['dbscan_outlier'] == 1),
        np.sum(soft['ocsvm_outlier'] == 1),
    ]
}
summary_df = pd.DataFrame(outlier_summary)
summary_df

,Method,Outliers,Inliers
0,Isolation Forest,51,2021
1,DBSCAN,0,2072
2,One-Class SVM,50,2022
